In [ ]:
from util.dataset import load_dataset
from data_pipeline.data_generator import LatencyEstimator
import pandas as pd
from analysis.binary_gates import latency_binary_gate, accumulate_latency
pd.options.display.max_rows = 999

In [ ]:
le = LatencyEstimator(
    block_type=0, input_channel=512, num_layers=5, dataset=load_dataset(batch_size=64)
)
le.model.init_arch_params()

## How many times do you need to estimate latency when resetting the binary gate?

In [ ]:
iter_list = [50,40,30,20,10,5]
latency_list = list(map(lambda n: latency_binary_gate(le, n), iter_list))

In [ ]:
from functools import reduce
describe = map(lambda x: x.describe().to_frame().T.rename(index={'latency': len(x)}), latency_list)
combined = reduce(lambda x, y: pd.concat([x, y]), describe)

In [ ]:
combined

In [ ]:
latency_list[2].describe().to_frame().T

###  is 10 times proper

In [ ]:
iter_list2 = [10]*100
count = 0

In [ ]:
from util.outlier import cut_outlier

def func(x):
    l = latency_binary_gate(le, x)
    cut = cut_outlier(l, min_border=0.2, max_border=0.8)
    mid = l.quantile(0.4)
    mean = cut.mean()
    ratio = ((abs(l.quantile(0.4) - mean))/mean)*100  # percent
    print(mid, mean, ratio)
    return [mid, mean, ratio]
    
result = list(map(func, iter_list2))

In [ ]:
df = pd.DataFrame(result, columns=['0.4', 'mean', 'ratio'])

In [ ]:
df.describe()

In [ ]:
err = abs(df['0.4'] - df['mean'])

In [ ]:
err.describe()

## Accumulate the latency measured each time the binary gate is reset.

In [ ]:
cumulative_avg, cumulative_err, cumulative_latency = accumulate_latency(le, max_reset_times=10000)

In [ ]:
 cumulative_latency.plot()

In [ ]:
cumulative_avg.plot()

In [ ]:
cumulative_err.plot()

In [ ]:
test0 = cumulative_avg()

In [ ]:
cum0, err0, lat0= pd.Series(test0[0]), pd.Series(test0[1]), pd.Series(test0[2])

In [ ]:
lat0.plot()

In [ ]:
cum0.plot()

In [ ]:
err0.plot()

In [ ]:
test1 = cumulative_avg()

In [ ]:
cum, err, lat= pd.Series(test1[0]), pd.Series(test1[1]), pd.Series(test1[2])

In [ ]:
cum.plot()

In [ ]:
err.plot()

In [ ]:
lat.plot()

In [ ]:
test2 = cumulative_avg()

In [ ]:
cum2, err2, lat2= pd.Series(test2[0]), pd.Series(test2[1]), pd.Series(test2[2])

In [ ]:
cum2.plot()

In [ ]:
err2.plot()